In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
sns.set_theme(style="darkgrid")

In [2]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
folium.Icon(color='red', icon='')

In [3]:
divvy_jan = pd.read_csv("202301-divvy-tripdata.csv")
divvy_jan.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,Lincoln Ave & Fullerton Ave,TA1309000058,Hampden Ct & Diversey Ave,202480.0,41.924074,-87.646278,41.930000,-87.640000,member
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member


In [4]:
# creating a dataset with with the most frequent start and end statons used by users. 

station_count_users = divvy_jan.groupby("member_casual")[["start_station_name", "end_station_name", "start_lat", "start_lng", "end_lat", "end_lng"]].value_counts().reset_index()
station_count_users.columns = ["member_casual","start_station_name","end_station_name","start_lat", "start_lng", "end_lat","end_lng", "total"]

In [5]:
#Filtering by the 10 most used stations by users.
station_count_casual = station_count_users.query("member_casual == 'casual'").head(10).reset_index()
station_count_member = station_count_users.query("member_casual == 'member'").head(10).reset_index()

#creating a geodataset to ploting a map
station_count_casual_geo = gpd.GeoDataFrame(station_count_casual, 
                                            geometry= gpd.points_from_xy(station_count_casual.start_lat, station_count_casual.start_lng))

#set the geocoder parameter
station_count_casual_geo.crs = {'init': 'epsg:4326'}

#print the new dataset
station_count_casual_geo

In [6]:
map = folium.Map(location= [station_count_casual.start_lat.mean(), station_count_casual.start_lng.mean()],
                 zoom_start= 16,
                 control_scale= True)
folium.Marker(location= [station_count_casual.start_lat.mean(), station_count_casual.start_lng.mean()],
              icon=folium.Icon(color='red', icon='')).add_to(map).add_to(map)
map

In [7]:
#ploting the map with folium lib
map_divvy_casual = folium.Map(location=[station_count_casual.start_lat.mean(), station_count_casual.start_lng.mean()])
for idx, row in station_count_casual.iterrows():
    folium.Marker([row['start_lat'], row['start_lng']]).add_to(map_divvy_casual)


# Show the map
map_divvy_casual

In [8]:
#ploting the map with folium lib
map_divvy_member = folium.Map(location=[station_count_member.start_lat.mean(), station_count_member.start_lng.mean()])
for idx, row in station_count_member.iterrows():
    folium.Marker([row['start_lat'], row['start_lng']]).add_to(map_divvy_member)

#Setup the content of the popup
purpose_colour = {'WILDCAT':'red', 'APPRAISAL':'green', 'WILDCAT-CCS':'blue'}
for i,row in station_count_member.iterrows():
    #Setup the content of the popup
    iframe = folium.IFrame(f'Well Name: {str(row["Well Name"])} \n Purpose: {str(row["Purpose"])}')
    
    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=300, max_width=300)
    
    try:
        icon_color = purpose_colour[row['Purpose']]
    except:
        #Catch nans
        icon_color = 'gray'
    
    #Add each row to the map
    folium.Marker(location=[row['start_lat'],row['start_lng']],
                  popup = popup, 
                  icon=folium.Icon(color=icon_color, icon='')).add_to(map)

# Show the map
map_divvy_member

KeyError: 'Well Name'